## Libraries

In [1]:
from tensorflow.keras.layers import LSTM, Dense, Embedding, Conv1D, Dropout, SpatialDropout1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk
import re

2025-07-16 16:44:46.927077: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 16:44:47.196957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752680687.313148    2954 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752680687.347756    2954 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752680687.555519    2954 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
df = pd.read_csv('twitter30k.csv')
df.head()

,twitts,sentiment
0,@robbiebronniman Sounds like a great night.,1
1,Damn the person who stolde my wallet !!!!! Ma...,1
2,Greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars Pretty pretty pretty please, pak...",0


In [3]:
df.shape

(30000, 2)

In [4]:
x = df['twitts'].tolist()
y = df['sentiment']

In [5]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [6]:
x_train = []
for sent in x:
    sent = re.sub("[^a-zA-Z]", " ", sent)
    sent = sent.lower().split()
    sent = [lemmatizer.lemmatize(word) for word in sent if word not in set(stop_words)]
    sent = " ".join(sent)
    x_train.append(sent)

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [8]:
vocab_size = len(tokenizer.word_index) + 1 

In [9]:
encoded_texts = tokenizer.texts_to_sequences(x_train)

In [10]:
testing_list = []
for l in x_train:
    testing_list.append(len(l.split(' ')))

In [11]:
max_sentence_length = max(testing_list)

In [12]:
padded_x = pad_sequences(encoded_texts, maxlen=max_sentence_length, padding='post', truncating='post')

In [13]:
x_train, x_test, y_train, y_test = train_test_split(padded_x, y, test_size=0.15, random_state=42)

In [14]:
embedding_features = 300

## Building the model

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_features, input_length=max_sentence_length))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(64, recurrent_activation='relu', recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(32, recurrent_activation='relu', recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/home/amina/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-07-16 16:45:03.566815: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
history = model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 157s 182ms/step - accuracy: 0.5043 - loss: 0.6931 - val_accuracy: 0.5013 - val_loss: 0.6931
Epoch 2/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 192s 166ms/step - accuracy: 0.4977 - loss: 0.6932 - val_accuracy: 0.4987 - val_loss: 0.6932
Epoch 3/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 140s 176ms/step - accuracy: 0.4978 - loss: 0.6932 - val_accuracy: 0.4987 - val_loss: 0.6932
Epoch 4/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 139s 175ms/step - accuracy: 0.4953 - loss: 0.6932 - val_accuracy: 0.4987 - val_loss: 0.6932
Epoch 5/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 140s 175ms/step - accuracy: 0.4952 - loss: 0.6932 - val_accuracy: 0.4987 - val_loss: 0.6932
Epoch 6/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 140s 176ms/step - accuracy: 0.4999 - loss: 0.6932 - val_accuracy: 0.5013 - val_loss: 0.6931
Epoch 7/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 155s 192ms/step - accuracy: 0.5031 - loss: 0.6932 - val_accuracy: 0.4987 - val_loss: 0.6931
Epoch 8/10
797/797 ━━━━━━━━━━━━━━━━━━━━ 136s 170ms/step - accuracy: 0.5011 -